In [34]:
import sys
from os.path import dirname
sys.path.append("./lattice-estimator")

from estimator import *

Logging.set_level(Logging.LEVEL0)


a = 12 # rank
secpar = 2^7 
m = 2^20 # wit dim
B = 2^25 # norm 
q = 2^63 
N = 128 # degree
gamma = 2^7 # challenge set use biased bin challenges, also expansion factor or ring
L = 8 # instances
ring_split = 2 # assume ring splits into quadratic extensions, each NTT slor has enough entropy https://eprint.iacr.org/2025/1080.pdf

Zq = log(q,2)
Rq = log(q,2) * N

# 1. range check
# degree-2 sumcheck
claims = Rq * L
sumcheck = Zq * log(m,2) * 2 
reduced_rhs = 2 * Rq * L 

step_1 = claims + sumcheck + reduced_rhs

# 2. projection
step_2 = Rq * (L + 1)

# 3. homogenization (assume t = 4 for R1CS)

# degree-2 sumcheck
sumcheck = Zq * log(m,2) * 2 * ring_split
new_rhs = 4 * (L + 1) * Rq
step_3 = new_rhs + sumcheck

# 4. decomp 

step_4 = Rq * a * (L + 1) # (into 2 factors is enough)


print("costs: ", float(step_1 + step_2 + step_3 + step_4) / 1024 / 8 , "KB")

costs:  175.1572265625 KB


In [35]:
# norm does not grow check

norm_init = B
projected_norm = B * sqrt(337)
folded_norm = B * L 

decomp_base = sqrt(folded_norm) 
def decomp_norm(b): # KLNO24
    return sqrt(m) * sqrt(b) # translation between inf and 2 norms

decomposed_norm = max(decomp_norm(projected_norm), decomp_norm(folded_norm))

assert norm_init > decomposed_norm

In [36]:
# extraction check

composed_norm = decomposed_norm*decomp_base + decomposed_norm

extracted_norm_numerator = 2 * composed_norm 
extrancted_slack = 2 * gamma

sis_break = 2 * extracted_norm_numerator * extrancted_slack # the worst moment of the norm

params = SIS.Parameters(n=N * a, m=m * N, q=q, length_bound=sis_break, norm=2)
costs = SIS.estimate(params) 



lattice  :: rop: ≈2^135.3, red: ≈2^135.3, δ: 1.004234, β: 366, d: 3982, tag: euclidean
